In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import scipy.integrate as inte
import scipy.optimize as opti
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import row

output_notebook(hide_banner=True)
%matplotlib inline

## Système 1D

# Cas 1 : $u=u_0 \sin(\omega t)$

Après adimensionnement de l'équation originale, on obtient le système suivant :

$$
  \lbrace
        \begin{array}{ccc}
            \frac{dx}{dt} & = & v  \\
            \frac{dv}{dt} & = & - \frac{v - u_0 sin(t)}{St_{\omega}}
        \end{array}
$$

# Cas 2 : $u = u_0 \sin(\kappa x)$

Après adimensionnement de l'équation originale, on obtient le système suivant :

$$
  \lbrace
        \begin{array}{ccc}
            \frac{dx}{dt} & = & v  \\
            \frac{dv}{dt} & = & - \frac{v - \sin(x)}{St_{\kappa}}
        \end{array}
$$

In [2]:
#définition des paramètres caractéristiques :
u0 = 1/4 #vitesse caractéristique
k = 1 #kappa
tau = 1 #temps caractéristique
St_k = u0 *k * tau #nombre de Stokes 
print("St_k : ", St_k)

#définition de l'espace d'évolution de la particule
tini = 0
tend = 100
x_min = 0
x_max = 10
x0 = 0.01 #position initiale de la particule
v0 = 0 #vitesse initiale
xini = (x0, v0)
tol = 1.e-8

def f_2(t, x):
  return [x[1], - (x[1] - np.sin(x[0])) / St_k]

sol = inte.solve_ivp(f_2, (tini, tend), xini, rtol=tol, atol=tol)

St_k :  0.25


In [3]:
#affichage de l'évolution de la solution au cours du temps
fig = figure(x_range=(tini, tend), width=950, height=350, title="Evolution de la position de la particule au cours du temps")
fig.x(sol.t, sol.y[0], legend_label = "Position au cours du temps")
show(fig)

#tracé de la trajectoire dans l'espace des phases
fig_phase = figure(x_range=(x_min, x_max), width=950, height=350, title="Evolution de la trajectoire au cours du temps dans l'espace des phases")
fig_phase.x(sol.y[0,0], sol.y[1,0], color = "black", line_width = 3)
fig_phase.x(sol.y[0], sol.y[1], legend_label = "Trajectoire dans l'espace des phases")
fig_phase.line(np.linspace(x_min, x_max, 1000), np.sin(np.linspace(x_min, x_max, 1000)), color = "black", legend_label = "Vitesse du fluide")
for i in range(int(x_min // np.pi), int(x_max // np.pi) + 1) :
  if i % 2 == 0 :
    fig_phase.x(i * np.pi, 0, color = "red", line_width = 5)
  else :
    fig_phase.x(i * np.pi, 0, color = "green", line_width = 5)
show(fig_phase)

# Cas 3 : $u =u_0 \sin(\omega t + \kappa x)$

In [4]:
#définition des paramètres caractéristiques :
u0 = 2 #vitesse caractéristique
w = 1 #omega
k = 1 #kappa
tau = 1 #temps caractéristique
St_w = w * tau #nombre de Stokes pour omega
St_k = u0 *k * tau #nombre de Stokes pour kappa
print("St_w : ", St_w, ", St_k : ", St_k)

#définition de l'espace d'évolution de la particule
tini = 0
tend = 100
x_min = 0
x_max = 10
x0 = 0.01 #position initiale de la particule
v0 = 0 #vitesse initiale
xini = (x0, v0)
tol = 1.e-8

def f_2(t, x):
  return [x[1], 1 / St_w * (St_k / St_w * np.sin(x[0]) + 1 - x[1])]

sol = inte.solve_ivp(f_2, (tini, tend), xini, rtol=tol, atol=tol)

St_w :  1 , St_k :  2


In [5]:
#affichage de l'évolution de la solution au cours du temps
fig = figure(x_range=(tini, tend), width=950, height=350, title="Evolution de la position de la particule au cours du temps")
fig.x(sol.t, sol.y[0], legend_label = "Position au cours du temps")
show(fig)

#tracé de la trajectoire dans l'espace des phases
fig_phase = figure(x_range=(x_min, x_max), width=950, height=350, title="Evolution de la trajectoire au cours du temps dans l'espace des phases")
fig_phase.x(sol.y[0,0], sol.y[1,0], color = "black", line_width = 3)
fig_phase.x(sol.y[0], sol.y[1], legend_label = "Trajectoire dans l'espace des phases")
fig_phase.line(np.linspace(x_min, x_max, 1000), np.sin(np.linspace(x_min, x_max, 1000)), color = "black", legend_label = "Vitesse du fluide")
for i in range(int(x_min // np.pi), int(x_max // np.pi) + 1) :
  if i % 2 == 0 :
    fig_phase.x(i * np.pi, 0, color = "red", line_width = 5)
  else :
    fig_phase.x(i * np.pi, 0, color = "green", line_width = 5)
show(fig_phase)

In [6]:
#animation de la trajectoire de la particule au cours du temps
fig, ax = plt.subplots()
ax.plot(np.linspace(x_min, x_max, 1000), np.sin(np.linspace(x_min, x_max, 1000), color = "black", label = "Vitesse") #ax.streamplot(np.linspace(0,1,nx, endpoint=True),np.linspace(0,AR,ny, endpoint=True), liste_u[1], liste_v[1], color=liste_u[1], linewidth=2, cmap='autumn')
line, = ax.plot([], [], lw=2) 

# initialization function 
def init(): 
    # creating an empty plot/frame 
    line.set_data([], []) 
    return line, 

# lists to store x and y axis points 
xdata, ydata = [], [] 

# animation function 
def animate(i): 

    # appending new points to x, y axes points list 
    xdata.append(sol.y[0,i]) 
    ydata.append(sol.y[0,i]) 
    line.set_data(xdata, ydata) 
    return line, 

# animate
anim = animation.FuncAnimation(fig, make_frame, interval = 80, repeat = True);

SyntaxError: invalid syntax (<ipython-input-6-5f39aa5dd8f6>, line 4)